In [1]:
import pandas as pd

# Load the dataset
file_path = "/content/Disaster_Dataset.csv"  # Adjust the path if necessary

df = pd.read_csv(file_path)  # Read CSV file into a DataFrame

# Filter the dataset to keep only rows where designatedArea ends with "(County)"
df_filtered = df[df['designatedArea'].str.endswith("(County)")]

# Select relevant columns for analysis
columns_to_keep = [
    "Risk_Categories", "state", "declarationYear", "incidentType",
    "declarationTitle", "designatedArea", "disaster_count"
]
df_filtered = df_filtered[columns_to_keep]

# Pivot the dataset so each county has one row, with multiple incidents as separate columns
df_pivot = df_filtered.pivot_table(
    index=["Risk_Categories", "state", "declarationYear", "designatedArea"],
    columns="incidentType",
    values="disaster_count",
    aggfunc="sum",  # Summing disaster counts for each incident type
    fill_value=0  # Replace missing values with 0
).reset_index()

# Rename columns for clarity
df_pivot.columns.name = None  # Remove column index name
df_pivot = df_pivot.rename(columns=lambda x: x.replace(" ", "_"))  # Replace spaces with underscores

# Display the cleaned dataset in Colab
from IPython.display import display
display(df_pivot)  # Show the transformed dataset


,Risk_Categories,state,declarationYear,designatedArea,Coastal_Storm,Dam/Levee_Break,Earthquake,Fire,Flood,Hurricane,Mud/Landslide,Other,Severe_Ice_Storm,Severe_Storm,Snowstorm,Straight-Line_Winds,Tornado,Tropical_Storm,Volcanic_Eruption,Winter_Storm
0,High Risk,AL,2014,Butler (County),0,0,0,0,0,0,0,0,0,167,0,0,0,0,0,0
1,High Risk,AL,2014,Lee (County),0,0,0,0,0,0,0,0,0,268,0,0,0,0,0,0
2,High Risk,AL,2014,Perry (County),0,0,0,0,0,0,0,0,0,232,0,0,0,0,0,0
3,High Risk,AL,2016,Butler (County),0,0,0,0,0,0,0,0,0,167,0,0,0,0,0,0
4,High Risk,AL,2016,Cherokee (County),0,0,0,0,0,0,0,0,0,185,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9271,Severe Risk,WV,2018,Jefferson (County),0,0,0,0,0,0,0,0,0,517,0,0,0,0,0,0
9272,Severe Risk,WV,2018,Lincoln (County),0,0,0,0,0,0,444,0,0,0,0,0,0,0,0,0
9273,Severe Risk,WV,2021,Lincoln (County),0,0,0,0,444,0,0,0,444,0,0,0,0,0,0,0
9274,Severe Risk,WV,2024,Clay (County),0,0,0,0,393,0,0,0,0,0,0,0,0,0,0,0


This code loads a dataset containing disaster declarations, filters it to include only county-level records, and selects relevant columns. It then pivots the dataset so that each county appears as a single row with separate columns for different incident types, summing the disaster counts accordingly. Finally, it renames columns for clarity and displays the cleaned dataset.

In [2]:
# Aggregate by summing incident counts and counting rows for disaster declarations
df_aggregated = (
    df_pivot.groupby(["Risk_Categories", "state", "designatedArea"], as_index=False)
    .agg({
        "declarationYear": "max",  # Use the latest declaration year
        "Coastal_Storm": "sum",
        "Dam/Levee_Break": "sum",
        "Earthquake": "sum",
        "Fire": "sum",
        "Flood": "sum",
        "Hurricane": "sum",
        "Mud/Landslide": "sum",
        "Other": "sum",
        "Severe_Ice_Storm": "sum",
        "Severe_Storm": "sum",
        "Snowstorm": "sum",
        "Straight-Line_Winds": "sum",
        "Tornado": "sum",
        "Tropical_Storm": "sum",
        "Volcanic_Eruption": "sum",
        "Winter_Storm": "sum",
    })
)

# Recalculate disaster_count based on the number of rows (declarations) for each county
original_disaster_counts = (
    df_filtered.groupby(["Risk_Categories", "state", "designatedArea"])
    .size()
    .reset_index(name="disaster_count")
)

# Merge the original disaster counts back into the aggregated data
df_aggregated = pd.merge(
    df_aggregated,
    original_disaster_counts,
    on=["Risk_Categories", "state", "designatedArea"],
    how="left"
)

# Display the updated dataset
from IPython.display import display
display(df_aggregated)  # Show the aggregated dataset


,Risk_Categories,state,designatedArea,declarationYear,Coastal_Storm,Dam/Levee_Break,Earthquake,Fire,Flood,Hurricane,...,Other,Severe_Ice_Storm,Severe_Storm,Snowstorm,Straight-Line_Winds,Tornado,Tropical_Storm,Volcanic_Eruption,Winter_Storm,disaster_count
0,High Risk,AL,Butler (County),2024,0,0,0,0,0,1169,...,0,0,501,0,0,0,0,0,0,10
1,High Risk,AL,Calhoun (County),2024,0,0,0,0,0,1004,...,0,0,502,0,0,0,0,0,0,6
2,High Risk,AL,Cherokee (County),2024,0,0,0,0,0,740,...,0,0,370,0,0,0,0,0,0,6
3,High Risk,AL,Fayette (County),2024,0,0,0,0,0,570,...,0,0,380,0,0,0,0,0,0,5
4,High Risk,AL,Greene (County),2024,0,0,0,0,0,1188,...,0,0,891,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613,Severe Risk,WV,Jackson (County),2016,0,0,0,0,1494,0,...,0,0,996,0,0,0,0,0,0,5
2614,Severe Risk,WV,Jefferson (County),2018,0,0,0,0,0,0,...,0,0,517,0,0,0,0,0,0,1
2615,Severe Risk,WV,Lincoln (County),2024,0,0,0,0,1332,0,...,0,444,1776,0,0,0,0,0,0,9
2616,Severe Risk,WV,Marion (County),2017,0,0,0,0,0,0,...,0,0,362,0,0,0,0,0,0,1


This code aggregates disaster data by summing incident counts for each county while retaining the most recent declaration year. It then recalculates the total number of disaster declarations per county and merges this count back into the aggregated dataset. Finally, the updated dataset is displayed for review.

In [3]:
# Deduplicate incidents based on disasterNumber, designatedArea, and incidentType
deduplicated_disasters = df_filtered.drop_duplicates(subset=["declarationTitle", "designatedArea", "incidentType"])

# Count unique disasters by incident type for each designatedArea
incident_type_counts = (
    deduplicated_disasters.groupby(["designatedArea", "incidentType"])
    .size()
    .unstack(fill_value=0)  # Create columns for each incident type
    .reset_index()
)

# Merge the corrected incident type counts with the aggregated dataset
final_corrected_dataset = pd.merge(
    df_aggregated.drop(columns=df_aggregated.columns.difference(["Risk_Categories", "state", "designatedArea", "disaster_count"])),
    incident_type_counts,
    on="designatedArea",
    how="left"
)

# Fill NaN values with 0 for incident type counts
final_corrected_dataset.fillna(0, inplace=True)

# Convert incident type columns to integers
incident_columns = list(incident_type_counts.columns.difference(["designatedArea"]))
final_corrected_dataset[incident_columns] = final_corrected_dataset[incident_columns].astype(int)

# Display the corrected dataset
from IPython.display import display
display(final_corrected_dataset)


,Risk_Categories,state,designatedArea,disaster_count,Coastal Storm,Dam/Levee Break,Earthquake,Fire,Flood,Hurricane,Mud/Landslide,Other,Severe Ice Storm,Severe Storm,Snowstorm,Straight-Line Winds,Tornado,Tropical Storm,Volcanic Eruption,Winter Storm
0,High Risk,AL,Butler (County),10,0,0,0,0,6,6,0,0,0,12,1,0,0,0,0,1
1,High Risk,AL,Calhoun (County),6,0,0,0,1,3,19,1,0,2,8,0,0,1,5,0,1
2,High Risk,AL,Cherokee (County),6,0,0,0,0,3,13,0,0,2,10,1,0,1,3,0,0
3,High Risk,AL,Fayette (County),5,0,0,0,0,5,5,0,0,2,10,1,0,1,1,0,0
4,High Risk,AL,Greene (County),7,0,0,0,0,4,15,0,0,2,9,1,0,1,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613,Severe Risk,WV,Jackson (County),5,0,0,0,7,13,22,0,0,3,19,1,0,2,5,0,1
2614,Severe Risk,WV,Jefferson (County),1,0,0,0,7,16,19,0,0,3,21,1,0,1,7,0,2
2615,Severe Risk,WV,Lincoln (County),9,0,0,0,17,14,13,1,0,3,16,1,0,2,2,0,1
2616,Severe Risk,WV,Marion (County),1,0,0,0,4,5,20,0,0,3,17,0,0,3,6,0,1


This code removes duplicate disaster records based on `declarationTitle`, `designatedArea`, and `incidentType`, ensuring unique disaster counts per county. It then counts the number of unique disaster incidents for each incident type and merges these counts with the previously aggregated dataset. Missing values are replaced with zeros, and all incident type columns are converted to integers before displaying the corrected dataset.

In [5]:
# Group by state, designatedArea, and incidentType to count disasters accurately for all counties
incident_type_counts = (
    df_filtered.groupby(["state", "designatedArea", "incidentType"])
    .size()
    .unstack(fill_value=0)  # Create columns for each incident type
    .reset_index()
)

# Recalculate total incidents by summing across all incident types
incident_type_counts["total_incidents"] = incident_type_counts.drop(
    columns=["state", "designatedArea"]
).sum(axis=1)

# Merge recalculated incident type counts with the aggregated dataset
final_corrected_dataset = pd.merge(
    df_aggregated.drop(columns=df_aggregated.columns.difference(["Risk_Categories", "state", "designatedArea", "disaster_count"])),
    incident_type_counts,
    on=["state", "designatedArea"],
    how="left"
)

# Replace NaN values with 0 for incident type counts
final_corrected_dataset.fillna(0, inplace=True)

# Convert all incident type columns to integers
incident_columns = list(incident_type_counts.columns.difference(["state", "designatedArea", "total_incidents"]))
final_corrected_dataset[incident_columns] = final_corrected_dataset[incident_columns].astype(int)

# Verify that disaster_count matches total incidents for all counties
final_corrected_dataset["discrepancy"] = (
    final_corrected_dataset["disaster_count"] - final_corrected_dataset["total_incidents"]
)

# Identify rows with discrepancies (if any)
discrepancies = final_corrected_dataset[final_corrected_dataset["discrepancy"] != 0]
if not discrepancies.empty:
    print("Discrepancies found:")
    print(discrepancies)
else:
    print("No discrepancies found. Data is consistent.")

# Display the corrected dataset
from IPython.display import display
display(final_corrected_dataset.head())

# Save the final corrected dataset
final_corrected_dataset.to_csv("/content/Actual_Final_Corrected_Disaster_Dataset.csv", index=False)
print("File saved as Final_Corrected_Disaster_Dataset.csv.")


No discrepancies found. Data is consistent.


,Risk_Categories,state,designatedArea,disaster_count,Coastal Storm,Dam/Levee Break,Earthquake,Fire,Flood,Hurricane,...,Severe Ice Storm,Severe Storm,Snowstorm,Straight-Line Winds,Tornado,Tropical Storm,Volcanic Eruption,Winter Storm,total_incidents,discrepancy
0,High Risk,AL,Butler (County),10,0,0,0,0,0,7,...,0,3,0,0,0,0,0,0,10,0
1,High Risk,AL,Calhoun (County),6,0,0,0,0,0,4,...,0,2,0,0,0,0,0,0,6,0
2,High Risk,AL,Cherokee (County),6,0,0,0,0,0,4,...,0,2,0,0,0,0,0,0,6,0
3,High Risk,AL,Fayette (County),5,0,0,0,0,0,3,...,0,2,0,0,0,0,0,0,5,0
4,High Risk,AL,Greene (County),7,0,0,0,0,0,4,...,0,3,0,0,0,0,0,0,7,0


File saved as Final_Corrected_Disaster_Dataset.csv.


This code groups disaster data by `state`, `designatedArea`, and `incidentType` to count the number of disasters for each type at the county level. It then calculates the total number of incidents per county and merges this information with the aggregated dataset. Missing values are replaced with zeros, and all incident type columns are converted to integers. Finally, it checks for discrepancies between the total disaster count and the sum of individual incident types, displays the corrected dataset, and saves it as a CSV file.